In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import matplotlib.pyplot as plt 
import seaborn as sns


In [2]:
!pip3 install xgboost

In [3]:
pd.set_option('display.max_rows', 100)
df_x = pd.read_csv(r'/Users/tuongvan.nguyen/Documents/CS5228/cs5228-2021s1/Xtrain.csv',index_col='Id')
df_y = pd.read_csv(r'/Users/tuongvan.nguyen/Documents/CS5228/cs5228-2021s1/Ytrain.csv',index_col='Id')
# Load the SBA loan data and make a copy for exploration

df = pd.concat([df_x,df_y],axis=1)
df_used = df.copy(deep=True)
print("Shape of dataframe {}".format(df_used.shape))
print(df_used.columns)
df_used

Shape of dataframe (50000, 24)
Index(['Name', 'City', 'State', 'Zip', 'Bank', 'BankState', 'NAICS',
       'ApprovalDate', 'ApprovalFY', 'Term', 'NoEmp', 'NewExist', 'CreateJob',
       'RetainedJob', 'FranchiseCode', 'UrbanRural', 'RevLineCr', 'LowDoc',
       'DisbursementDate', 'DisbursementGross', 'BalanceGross', 'GrAppv',
       'SBA_Appv', 'ChargeOff'],
      dtype='object')


,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,...,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementDate,DisbursementGross,BalanceGross,GrAppv,SBA_Appv,ChargeOff
Id,,,,,,,,,,,,,,,,,,,,,
0,"NPE Realty, LLC",Pembroke Pines,FL,33027,"TD BANK, NATIONAL ASSOCIATION",DE,334516,16-Apr-10,2010,130,...,0,1,N,N,1-May-10,"$2,000,000.00",$0.00,"$2,000,000.00","$1,500,000.00",0
1,"KEVCO CONSTRUCTION, LLC",SANDY,UT,84094,ZIONS FIRST NATIONAL BANK,UT,236115,30-Jan-10,2010,33,...,0,1,N,N,1-Feb-10,"$17,000.00",$0.00,"$17,000.00","$17,000.00",1
2,EAST L A SHOE REPAIR,LOS ANGELES,CA,90022,BANK OF AMERICA NATL ASSOC,NC,811430,29-Jul-02,2002,60,...,1,1,Y,N,31-Aug-02,"$17,000.00",$0.00,"$17,000.00","$8,500.00",0
3,"RUSSLER, DANIEL C, MD",LODI,WI,53555,ASSOCIATED BANK NATL ASSOC,WI,0,10-Jun-94,1994,160,...,1,0,N,N,31-Jul-94,"$137,000.00",$0.00,"$137,000.00","$123,300.00",0
4,AFFORDABLE FAMILY DENISTRY LLC,LEBANON,MO,65536,COMMERCE BANK,MO,621210,6-Feb-06,2006,101,...,1,1,T,N,31-Mar-06,"$46,665.00",$0.00,"$25,000.00","$12,500.00",0
5,COPPER COIN DISCOUNT LIQUOR,PAGOSA SPRINGS,CO,81147,FIRST SOUTHWEST BANK,CO,445310,17-Jan-08,2008,272,...,1,2,0,N,31-May-08,"$400,000.00",$0.00,"$500,000.00","$375,000.00",1
6,NICHOLAS MIRABELLA AND PAUL ST,LAS VEGAS,NV,89116,WELLS FARGO BANK NATL ASSOC,SD,811310,6-Sep-02,2002,300,...,1,1,0,N,31-Oct-02,"$275,000.00",$0.00,"$275,000.00","$206,250.00",0
7,SA SA LLC,ELLICOTT CITY,MD,21043,HOWARD BANK,MD,423940,15-Jan-10,2010,106,...,0,2,N,N,1-Feb-10,"$75,000.00",$0.00,"$75,000.00","$67,500.00",1
8,MICHAEL D CAIN & JANET H CAIN,CONCORD,CA,94518,BANK OF AMERICA NATL ASSOC,NY,621210,2-Dec-99,2000,300,...,1,1,0,N,31-Dec-99,"$699,300.00",$0.00,"$699,300.00","$524,475.00",0


In [4]:
#Clean types 
df_used[['DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv']] = df_used[['DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv']].applymap(lambda x: x.strip().replace('$', '').replace(',', '').replace('.', '')).astype('int64')
df_used['ApprovalFY'] = df_used['ApprovalFY'].apply(lambda x: x.replace('A', '')).apply('int64')
df_used['Industry'] = df_used['NAICS'].astype('str').apply(lambda x: x[:2])
df_used['Industry'] = df_used['Industry'].map({
    '11': 'Ag/For/Fish/Hunt',
    '21': 'Min/Quar/Oil_Gas_ext',
    '22': 'Utilities',
    '23': 'Construction',
    '31': 'Manufacturing',
    '32': 'Manufacturing',
    '33': 'Manufacturing',
    '42': 'Wholesale_trade',
    '44': 'Retail_trade',
    '45': 'Retail_trade',
    '48': 'Trans/Ware',
    '49': 'Trans/Ware',
    '51': 'Information',
    '52': 'Finance/Insurance',
    '53': 'RE/Rental/Lease',
    '54': 'Prof/Science/Tech',
    '55': 'Mgmt_comp',
    '56': 'Admin_sup/Waste_Mgmt_Rem',
    '61': 'Educational',
    '62': 'Healthcare/Social_assist',
    '71': 'Arts/Entertain/Rec',
    '72': 'Accom/Food_serv',
    '81': 'Other_no_pub',
    '92': 'Public_Admin'
})

df_used = df_used[df_used['NewExist'].isin(['0','1'])]
df_used['NewExist'] =df_used['NewExist'].astype('int64')
df_used['IsFranchised'] = df_used.FranchiseCode
df_used[df_used['IsFranchised'] <= 1].IsFranchised = 0 
df_used[df_used['IsFranchised'] > 1].IsFranchised = 1
df_used = df_used[df_used['NewExist'].isin([1,2])]
df_used = df_used[df_used['UrbanRural'].isin([0,1,2])]
df_used = df_used[df_used['LowDoc'].isin(['Y','N'])]
df_used = df_used[df_used['RevLineCr'].isin(['Y','N'])]

df_used['LowDoc'] = df_used['LowDoc'].map({'Y': '1','N': '0'}).astype('int64')
df_used['RevLineCr'] = df_used['RevLineCr'].map({'Y': '1','N': '0'}).astype('int64')
df_used[['ApprovalDate', 'DisbursementDate']] = df_used[['ApprovalDate', 'DisbursementDate']].apply(pd.to_datetime)
df_used['DisFY'] = df_used['DisbursementDate'].map(lambda x: x.year).apply('int64')
df_used['ratioApproved'] = df_used['SBA_Appv'] / df_used['GrAppv']
df_used['ratioBankAmount'] = df_used['GrAppv'] / df_used['DisbursementGross']
df_used.loc[df_used['GrAppv'] > df_used['DisbursementGross'],'BalanceComp'] = 1
df_used.loc[df_used['GrAppv'] < df_used['DisbursementGross'],'BalanceComp'] = 2
df_used.loc[df_used['GrAppv'] == df_used['DisbursementGross'],'BalanceComp'] = 0
df_used.loc[df_used['BankState'] == df_used['State'],'FlagState'] = 1
df_used.loc[df_used['BankState'] != df_used['State'],'FlagState'] = 0
df_used['CreateJobPer'] = df_used['CreateJob'] / df_used['NoEmp']
df_used['RetainedJobPer'] = df_used['RetainedJob'] / df_used['NoEmp']
df_used['BalanceComp'].astype('int64')

#Advanced features: TO DO:
#-> risk per industry
#-> risk per year
#-> flags for recession 2007 - 2009 / 2001 


/Users/tuongvan.nguyen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/tuongvan.nguyen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/tuongvan.nguyen/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://

Id
0        0
1        0
2        0
9        0
10       2
15       0
16       0
17       0
20       0
21       0
22       2
24       0
30       0
32       0
33       2
34       0
35       0
36       0
39       2
40       0
42       0
43       0
45       0
46       1
47       0
48       0
50       0
51       0
52       0
53       2
54       1
57       0
59       0
61       0
62       0
64       0
65       0
67       0
68       0
74       2
76       0
78       2
80       0
82       2
84       0
85       0
86       2
90       0
95       0
97       2
        ..
49909    0
49910    0
49912    2
49913    2
49918    0
49919    0
49920    0
49922    0
49923    0
49924    2
49925    2
49926    2
49927    2
49929    2
49930    0
49931    0
49932    2
49935    0
49937    0
49938    0
49940    0
49941    0
49944    1
49948    0
49951    0
49953    0
49956    0
49960    0
49961    2
49963    2
49967    2
49969    0
49970    0
49971    1
49972    0
49973    2
49974    0
49976    0
49978    0
49980  

In [5]:
df_used.columns

Index(['Name', 'City', 'State', 'Zip', 'Bank', 'BankState', 'NAICS',
       'ApprovalDate', 'ApprovalFY', 'Term', 'NoEmp', 'NewExist', 'CreateJob',
       'RetainedJob', 'FranchiseCode', 'UrbanRural', 'RevLineCr', 'LowDoc',
       'DisbursementDate', 'DisbursementGross', 'BalanceGross', 'GrAppv',
       'SBA_Appv', 'ChargeOff', 'Industry', 'IsFranchised', 'DisFY',
       'ratioApproved', 'ratioBankAmount', 'BalanceComp', 'FlagState',
       'CreateJobPer', 'RetainedJobPer'],
      dtype='object')

# Creation of first dataset
Keep only raw features

In [6]:
df_used_no_enhanced_features = df_used[['Name', 'City', 'State', 'Zip', 'Bank', 'BankState','ApprovalFY', 'Term', 'NoEmp', 'NewExist', 'CreateJob','RetainedJob', 'UrbanRural', 'RevLineCr', 'LowDoc','DisFY', 'DisbursementGross', 'BalanceGross', 'GrAppv','SBA_Appv', 'ChargeOff', 'IsFranchised']]
df_used_no_enhanced_features.drop(['Name', 'Zip', 'City','Bank'],inplace=True,axis=1)

/Users/tuongvan.nguyen/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [7]:
# should fill up the empty BankState with smt?

In [8]:
df_used_no_enhanced_features.dropna(inplace=True)
#Get categorical data
df_used_no_enhanced_features['BankState'] = df_used_no_enhanced_features['BankState'].astype('category')
df_used_no_enhanced_features['State'] = df_used_no_enhanced_features['State'].astype('category')
print(df_used_no_enhanced_features)

/Users/tuongvan.nguyen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/tuongvan.nguyen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/tuongvan.nguyen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

      State BankState  ApprovalFY  Term  NoEmp  NewExist  CreateJob  \
Id                                                                    
0        FL        DE        2010   130     25         1          2   
1        UT        UT        2010    33      6         1          1   
2        CA        NC        2002    60      1         1          1   
9        OH        CA        2007    10     15         1          5   
10       CO        OH        2007    84      1         1          0   
15       MA        MA        1996   120    325         1          0   
16       IA        IA        1984    84     25         1          0   
17       WI        WI        2004   240     14         1          0   
20       FL        VA        2006    54      5         1          3   
21       GA        NC        2002    84      5         1          0   
22       WA        WA        1999    58      2         1          0   
24       IL        IL        1998    84      6         1          0   
30    

In [9]:
print(df_used_no_enhanced_features.dtypes)
df_used_no_enhanced_features

State                category
BankState            category
ApprovalFY              int64
Term                    int64
NoEmp                   int64
NewExist                int64
CreateJob               int64
RetainedJob             int64
UrbanRural              int64
RevLineCr               int64
LowDoc                  int64
DisFY                   int64
DisbursementGross       int64
BalanceGross            int64
GrAppv                  int64
SBA_Appv                int64
ChargeOff               int64
IsFranchised            int64
dtype: object


,State,BankState,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,UrbanRural,RevLineCr,LowDoc,DisFY,DisbursementGross,BalanceGross,GrAppv,SBA_Appv,ChargeOff,IsFranchised
Id,,,,,,,,,,,,,,,,,,
0,FL,DE,2010,130,25,1,2,0,1,0,0,2010,200000000,0,200000000,150000000,0,0
1,UT,UT,2010,33,6,1,1,3,1,0,0,2010,1700000,0,1700000,1700000,1,0
2,CA,NC,2002,60,1,1,1,1,1,1,0,2002,1700000,0,1700000,850000,0,1
9,OH,CA,2007,10,15,1,5,15,2,0,0,2007,10000000,0,10000000,5000000,1,0
10,CO,OH,2007,84,1,1,0,1,1,1,0,2008,2926200,0,2500000,1250000,0,0
15,MA,MA,1996,120,325,1,0,0,0,0,0,1996,10000000,0,10000000,8000000,0,1
16,IA,IA,1984,84,25,1,0,0,0,0,0,1984,9000000,0,9000000,8100000,1,25035
17,WI,WI,2004,240,14,1,0,14,2,0,0,2004,45900000,0,45900000,45900000,0,1
20,FL,VA,2006,54,5,1,3,5,1,0,0,2006,5000000,0,5000000,2500000,1,0


In [10]:
all_states = list(set(list(df_used_no_enhanced_features['State'].unique())) | set(list(df_used_no_enhanced_features['BankState'].unique())))
#print('nulls ', df_used_no_enhanced_features['BankState'].isna().sum() + df_used_no_enhanced_features['State'].isna().sum())
all_states_id = [i for i in range(len(all_states))]
dict_map =  dict(zip(all_states, all_states_id))
#print('dict_map', dict_map)
#print('bankstateid\n', df_used_no_enhanced_features['BankState']);
#print(df_used['BankState'].map(dict_map))
#print(df_used['BankState'].map(dict_map).isna().sum())
#print(df_used['BankState'].map(dict_map).dropna())
df_used['StateId'] = df_used['State'].map(dict_map).dropna().astype('int64')
df_used['BankStateId'] = df_used['BankState'].map(dict_map).dropna().astype('int64')
df_used['BankStateId'].dropna(inplace=True)
print('ID ',df_used['BankStateId'])
df_used_no_enhanced_features_final = df_used_no_enhanced_features.drop('State', axis=1)
df_used_no_enhanced_features_final = df_used_no_enhanced_features.drop('BankState', axis=1)
print('final', df_used_no_enhanced_features_final)

ID  Id
0        31.0
1         1.0
2        21.0
9        41.0
10       27.0
15       45.0
16       17.0
17        8.0
20       51.0
21       21.0
22        6.0
24       48.0
30       41.0
32       27.0
33       48.0
34       51.0
35       28.0
36       15.0
39       41.0
40       36.0
42       21.0
43       21.0
45       35.0
46       18.0
47       51.0
48       41.0
50        2.0
51       35.0
52        7.0
53       21.0
54       21.0
57       41.0
59       21.0
61        2.0
62       12.0
64       51.0
65       44.0
67       31.0
68       13.0
74        2.0
76       35.0
78        8.0
80       34.0
82       20.0
84       36.0
85        7.0
86        2.0
90       37.0
95       34.0
97       51.0
         ... 
49906    35.0
49909     2.0
49910    48.0
49912    34.0
49913     2.0
49918     2.0
49919     4.0
49920     2.0
49922    20.0
49923     6.0
49924    51.0
49925     2.0
49926    21.0
49927    34.0
49929    19.0
49930    34.0
49931    21.0
49932    48.0
49935     5.0
49937     2.0

In [20]:
# Scale the feature values prior to modeling
y = df_used_no_enhanced_features_final['ChargeOff']
X = df_used_no_enhanced_features_final.drop('ChargeOff', axis=1)
X = df_used_no_enhanced_features_final.drop('State', axis=1)
scale = StandardScaler()
X.drop('ChargeOff', axis=1, inplace=True)
print(X)
X_scaled = scale.fit_transform(X)
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.25)

       ApprovalFY  Term  NoEmp  NewExist  CreateJob  RetainedJob  UrbanRural  \
Id                                                                             
0            2010   130     25         1          2            0           1   
1            2010    33      6         1          1            3           1   
2            2002    60      1         1          1            1           1   
9            2007    10     15         1          5           15           2   
10           2007    84      1         1          0            1           1   
15           1996   120    325         1          0            0           0   
16           1984    84     25         1          0            0           0   
17           2004   240     14         1          0           14           2   
20           2006    54      5         1          3            5           1   
21           2002    84      5         1          0            5           1   
22           1999    58      2         1

In [12]:
y_train.values.ravel()

array([0, 0, 0, ..., 1, 1, 0])

In [32]:
import xgboost as xgb
model=xgb.XGBClassifier(random_state=2,learning_rate=0.02)


AttributeError: 'numpy.ndarray' object has no attribute 'as_matrix'

## Correlation Analysis (to be completed)

In [18]:
from sklearn.metrics import accuracy_score
accuracy_score(y_val.values, model.predict(X_val))

1.0

In [31]:
# now process the test data and predict
import csv

test_x = pd.read_csv(r'/Users/tuongvan.nguyen/Documents/CS5228/cs5228-2021s1/Xtest.csv',index_col='Id')
#print(test_x['ApprovalFY'])
#Clean types 
test_x[['DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv']] = test_x[['DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv']].applymap(lambda x: x.strip().replace('$', '').replace(',', '').replace('.', '')).astype('int64')
#test_x['ApprovalFY'] = test_x['ApprovalFY'].apply(lambda x: x.replace('A', '')).apply('int64')
test_x['Industry'] = test_x['NAICS'].astype('str').apply(lambda x: x[:2])
test_x['Industry'] = test_x['Industry'].map({
    '11': 'Ag/For/Fish/Hunt',
    '21': 'Min/Quar/Oil_Gas_ext',
    '22': 'Utilities',
    '23': 'Construction',
    '31': 'Manufacturing',
    '32': 'Manufacturing',
    '33': 'Manufacturing',
    '42': 'Wholesale_trade',
    '44': 'Retail_trade',
    '45': 'Retail_trade',
    '48': 'Trans/Ware',
    '49': 'Trans/Ware',
    '51': 'Information',
    '52': 'Finance/Insurance',
    '53': 'RE/Rental/Lease',
    '54': 'Prof/Science/Tech',
    '55': 'Mgmt_comp',
    '56': 'Admin_sup/Waste_Mgmt_Rem',
    '61': 'Educational',
    '62': 'Healthcare/Social_assist',
    '71': 'Arts/Entertain/Rec',
    '72': 'Accom/Food_serv',
    '81': 'Other_no_pub',
    '92': 'Public_Admin'
})
print(test_x['NewExist'].isna().sum())
print(test_x['UrbanRural'].isna().sum())
print(test_x['LowDoc'].isna().sum())
print(test_x['RevLineCr'].isna().sum())
print(test_x['BankState'].isna().sum())
print(test_x['State'].isna().sum())

test_x = test_x[test_x['NewExist'].isin(['0','1'])]
test_x['NewExist'] =test_x['NewExist'].astype('int64')
test_x['IsFranchised'] = test_x.FranchiseCode
test_x[test_x['IsFranchised'] <= 1].IsFranchised = 0 
test_x[test_x['IsFranchised'] > 1].IsFranchised = 1
test_x['IsFranchised'].fillna(0)
test_x = test_x[test_x['NewExist'].isin([1,2])]
test_x = test_x[test_x['UrbanRural'].isin([0,1,2])]
test_x = test_x[test_x['LowDoc'].isin(['Y','N'])]
test_x = test_x[test_x['RevLineCr'].isin(['Y','N'])]

test_x['LowDoc'] = test_x['LowDoc'].map({'Y': '1','N': '0'}).astype('int64')
test_x['RevLineCr'] = test_x['RevLineCr'].map({'Y': '1','N': '0'}).astype('int64')
test_x[['ApprovalDate', 'DisbursementDate']] = test_x[['ApprovalDate', 'DisbursementDate']].apply(pd.to_datetime)
test_x['DisFY'] = test_x['DisbursementDate'].map(lambda x: x.year).apply('int64')
test_x['ratioApproved'] = test_x['SBA_Appv'] / test_x['GrAppv']
test_x['ratioBankAmount'] = test_x['GrAppv'] / test_x['DisbursementGross']
test_x.loc[test_x['GrAppv'] > test_x['DisbursementGross'],'BalanceComp'] = 1
test_x.loc[test_x['GrAppv'] < test_x['DisbursementGross'],'BalanceComp'] = 2
test_x.loc[test_x['GrAppv'] == test_x['DisbursementGross'],'BalanceComp'] = 0
test_x.loc[test_x['BankState'] == test_x['State'],'FlagState'] = 1
test_x.loc[test_x['BankState'] != test_x['State'],'FlagState'] = 0
test_x['CreateJobPer'] = test_x['CreateJob'] / test_x['NoEmp']
test_x['RetainedJobPer'] = test_x['RetainedJob'] / test_x['NoEmp']
test_x['BalanceComp'].astype('int64')
test_x[test_x['ApprovalFY']=='1976A']=1976
df_used_no_enhanced_features_1 = test_x[['Name', 'City', 'State', 'Zip', 'Bank', 'BankState','ApprovalFY', 'Term', 'NoEmp', 'NewExist', 'CreateJob','RetainedJob', 'UrbanRural', 'RevLineCr', 'LowDoc','DisFY', 'DisbursementGross', 'BalanceGross', 'GrAppv','SBA_Appv', 'IsFranchised']]
df_used_no_enhanced_features_1.drop(['Name', 'Zip', 'City','Bank'],inplace=True,axis=1)
df_used_no_enhanced_features_1.dropna(inplace=True)
#Get categorical data
df_used_no_enhanced_features_1['BankState'] = df_used_no_enhanced_features_1['BankState'].astype('category')
df_used_no_enhanced_features_1['State'] = df_used_no_enhanced_features_1['State'].astype('category')

all_states_1 = list(set(list(df_used_no_enhanced_features_1['State'].unique())) | set(list(df_used_no_enhanced_features_1['BankState'].unique())))
#print('nulls ', df_used_no_enhanced_features['BankState'].isna().sum() + df_used_no_enhanced_features['State'].isna().sum())
all_states_id_1 = [i for i in range(len(all_states_1))]
dict_map_1 =  dict(zip(all_states_1, all_states_id_1))
#print('dict_map', dict_map)
#print('bankstateid\n', df_used_no_enhanced_features['BankState']);
#print(df_used['BankState'].map(dict_map))
#print(df_used['BankState'].map(dict_map).isna().sum())
#print(df_used['BankState'].map(dict_map).dropna())
test_x['StateId'] = test_x['State'].map(dict_map_1).dropna().astype('int64')
test_x['BankStateId'] = test_x['BankState'].map(dict_map_1).dropna().astype('int64')
test_x['BankStateId'].dropna(inplace=True)

df_used_no_enhanced_features_final_1 = df_used_no_enhanced_features_1.drop('State', axis=1)
df_used_no_enhanced_features_final_1 = df_used_no_enhanced_features_1.drop('BankState', axis=1)

X1 = df_used_no_enhanced_features_final_1.drop('State', axis=1)
print(X1['IsFranchised'].isna().sum())

print(X1.columns)
print(X.columns)
scale_1 = StandardScaler()
X_scaled_1 = scale.fit_transform(X1)
res = model.predict(X_scaled_1)
data=[['Id', 'ChargeOff']]
print(len(res))
for i, val in enumerate(res):
    data.append([i,val])

with open(r'/Users/tuongvan.nguyen/Documents/CS5228/cs5228-2021s1/output.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in data:
        csvwriter.writerow(row)
#print(model.predict(X_scaled_1))


/Users/tuongvan.nguyen/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


14
0
463
341
112
4


/Users/tuongvan.nguyen/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/Users/tuongvan.nguyen/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/Users/tuongvan.nguyen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-cop

0
Index(['ApprovalFY', 'Term', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob',
       'UrbanRural', 'RevLineCr', 'LowDoc', 'DisFY', 'DisbursementGross',
       'BalanceGross', 'GrAppv', 'SBA_Appv', 'IsFranchised'],
      dtype='object')
Index(['ApprovalFY', 'Term', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob',
       'UrbanRural', 'RevLineCr', 'LowDoc', 'DisFY', 'DisbursementGross',
       'BalanceGross', 'GrAppv', 'SBA_Appv', 'IsFranchised'],
      dtype='object')
50297


In [ ]:
df_correlation  = df_used.dropna()
df_correlation

In [ ]:
# Correlation Matrix

cor_fig, cor_ax = plt.subplots(figsize=(15, 10))
corr_matrix = df_correlation.corr()
cor_ax = sns.heatmap(corr_matrix, annot=True)
plt.xticks(rotation=30, horizontalalignment='right', fontsize=8)
plt.yticks(fontsize=8)
plt.show()

In [ ]:
#Nan values cleanings 
#We will not use these fields:

df_copy.drop(subset=['Name','City','Bank', 'BankState', 'ApprovalDate','Term', 'LowDoc', 'DisbursementDate', 'MIS_Status'], inplace=True)
df_used.isnull().sum()

In [ ]:

df_copy.isnull().sum()